### Code

#### Import Necessary Packages

In [1]:
%pip install --q pandas
%pip install --q numpy
%pip install --q matplotlib
%pip install --q seaborn
%pip install --q scikit-learn
%pip install --q xgboost

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### Load Dataset

In [2]:
projects_df = pd.read_csv('projects.csv')
outcomes_df = pd.read_csv('outcomes.csv')

# Merge datasets on project ID
df = pd.merge(projects_df, outcomes_df[['projectid', 'fully_funded']], on='projectid', how='inner')

# Display basic information
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
df.head()

Dataset shape: (619326, 36)

First few rows:


,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,fully_funded
0,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,...,moderate poverty,Grades 3-5,30.0,444.36,522.78,7.0,f,f,2013-12-31,t
1,33d59ac771b80222ad63ef0f4ac47ade,de83b4c1f6428a15032c207c1d5e572a,d91a805b213bf74ae77b94e0de2b73ad,1.601530e+11,43.501154,-112.056780,Idaho Falls,ID,83402.0,urban,...,high poverty,Grades 3-5,30.0,233.24,274.40,30.0,f,f,2013-12-31,f
2,1a3aaeffc56dd2a421e37d8298024c0a,f4c9ed095b85458dcf858e25f203af00,9310d3eb447a4e46bc5fc31ed007ceac,3.302610e+11,42.888244,-71.320224,Derry,NH,3038.0,suburban,...,moderate poverty,Grades 6-8,30.0,285.09,335.40,230.0,f,f,2013-12-31,f
3,33aa19ee4da4c5adf47d0dfb84fab5ef,17768031eb40de8d4497dbb54df48742,9ac70da58322783f82152eecc140a812,5.103240e+11,37.476158,-77.488397,Richmond,VA,23224.0,urban,...,highest poverty,Grades PreK-2,30.0,232.94,274.05,18.0,f,f,2013-12-31,f
4,e31c0ea8b68f404699dfb0d39e9bc99b,0f1bc5b4700fd33383be104442660178,cb9f688cf59e3ee22a087d616ca8f5d7,1.709930e+11,41.952851,-87.650233,Chicago,IL,60613.0,urban,...,highest poverty,Grades 6-8,30.0,513.41,604.01,70.0,t,f,2013-12-31,t


#### Initial EDA

In [3]:
# Initial data exploration
print("Missing values per column:")
print(df.isnull().sum())
print("\nData types:")
print(df.dtypes)

print("\nTarget variable distribution:")
print(df['fully_funded'].value_counts(normalize=True))

# Basic statistics
print("\nNumerical features statistics:")
print(df.describe())

Missing values per column:
projectid                                      0
teacher_acctid                                 0
schoolid                                       0
school_ncesid                              38848
school_latitude                                0
school_longitude                               0
school_city                                    0
school_state                                   0
school_zip                                     4
school_metro                               75488
school_district                              922
school_county                                 17
school_charter                                 0
school_magnet                                  0
school_year_round                              0
school_nlns                                    0
school_kipp                                    0
school_charter_ready_promise                   0
teacher_prefix                                 0
teacher_teach_for_america                 

#### Data Preprocessing

In [4]:
def preprocess_data(df, columns_to_encode):
    """
    Preprocess data with one-hot encoding for specified columns
    """    
    # Create a copy to avoid modifying original dataframe
    processed_df = df.copy()

    # Drop all rows with missing values (TODO reconsider, but I think we have enough data to do this)
    processed_df = processed_df.dropna()
    
    # One-hot encode specified columns
    for column in columns_to_encode:
        if column in processed_df.columns:
            # Create dummy variables
            dummies = pd.get_dummies(
                processed_df[column], 
                prefix=column.lower().replace(' ', '_'),
                drop_first=True
            ).astype(int)
            
            # Drop original column and add encoded columns
            processed_df = processed_df.drop(column, axis=1)
            processed_df = pd.concat([processed_df, dummies], axis=1)
            
            print(f"Encoded {column} into {dummies.shape[1]} dummy variables")
    
    bool_columns = [
        'fully_funded',
        'school_charter',
        'school_magnet',
        'school_year_round',
        'school_nlns',
        'school_kipp',
        'school_charter_ready_promise',
        'teacher_teach_for_america',
        'teacher_ny_teaching_fellow',
        'eligible_double_your_impact_match',
        'eligible_almost_home_match',
        'great_chat',
        'at_least_1_teacher_referred_donor',
        'at_least_1_green_donation',
        'three_or_more_non_teacher_referred_donors',
        'one_non_teacher_referred_donor_giving_100_plus',
        'donation_from_thoughtful_donor'
    ]
    
    for col in bool_columns:
        if col in processed_df.columns:
            processed_df[col] = processed_df[col].map({'t': 1, 'f': 0})

    return processed_df

# Apply preprocessing with specific columns
try:
    print("Starting preprocessing...")
    
    columns_to_encode = [
        'primary_focus_area',
        'school_metro',
        'grade_level',
        'poverty_level',
        'resource_type'
    ]
    
    # Process the data
    processed_df = preprocess_data(df, columns_to_encode)
    
    # Display sample of processed data
    print("\nSample of processed data (first 5 rows):")
    display(processed_df.head())
    
    # Show shape before and after
    print(f"\nOriginal shape: {df.shape}")
    print(f"Processed shape: {processed_df.shape}")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

Starting preprocessing...
Encoded primary_focus_area into 6 dummy variables
Encoded school_metro into 2 dummy variables
Encoded grade_level into 3 dummy variables
Encoded poverty_level into 3 dummy variables
Encoded resource_type into 5 dummy variables

Sample of processed data (first 5 rows):


,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_district,...,grade_level_Grades 9-12,grade_level_Grades PreK-2,poverty_level_highest poverty,poverty_level_low poverty,poverty_level_moderate poverty,resource_type_Other,resource_type_Supplies,resource_type_Technology,resource_type_Trips,resource_type_Visitors
0,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,Elgin School District U-46,...,0,0,0,0,1,1,0,0,0,0
2,1a3aaeffc56dd2a421e37d8298024c0a,f4c9ed095b85458dcf858e25f203af00,9310d3eb447a4e46bc5fc31ed007ceac,3.302610e+11,42.888244,-71.320224,Derry,NH,3038.0,School Administrative Unit 10,...,0,0,0,0,1,0,0,1,0,0
6,a4b234feb2b72921ed59850d5c873d62,620982375045fa11d872702f26ab98bb,da1985df161ba5c3842fc99579b6e4b1,2.102990e+11,38.249919,-85.722310,Louisville,KY,40206.0,Jefferson Co School District,...,0,1,1,0,0,0,0,1,0,0
7,0ff5dec32bf793243a8b0b2c023a81f0,ec5b110df87bd511b508961676d08b6c,72e2b0cb2eecfdc37e67e0eaf10da07b,1.201980e+11,30.507978,-86.132003,Freeport,FL,32439.0,Walton Co School District,...,0,1,1,0,0,0,0,1,0,0
9,72d58b8e22adbe6e6aab65d8b2ab4973,0793851e2f40ecc28e40513622aea670,a180e99ef60be5a9c5f96e21f9868f13,5.606090e+11,44.097156,-104.622147,Upton,WY,82730.0,Weston Co School District 7,...,0,1,0,0,1,0,1,0,0,0



Original shape: (619326, 36)
Processed shape: (340210, 50)


In [5]:
# # TODO learn feature selection

# df_heatmap = processed_df.drop(columns=['school_ncesid', 'school_latitude', 'school_longitude'])
# df_heatmap = df_heatmap.drop(columns=['total_price_excluding_optional_support', 'total_price_including_optional_support',
#                                       'fulfillment_labor_materials'])

# # Create correlation heatmap
# plt.figure(figsize=(15, 10))

# # Select numerical columns and convert 't'/'f' to 1/0 for fully_funded
# numerical_df = df_heatmap.select_dtypes(include=['int64', 'float64']).copy()

# # Create correlation matrix
# corr_matrix = numerical_df.corr()

# mask = np.tril(np.ones_like(corr_matrix, dtype=bool), k=-1)

# # Create heatmap
# sns.heatmap(
#     corr_matrix, mask=mask, #annot=True,fmt='.2f',
#     cmap='PuOr', center=0, square=True
#     )

# plt.title('Correlation Heatmap of Numerical Features')
# plt.tight_layout()
# plt.show()

#### Feature Selection and Train/Test

In [5]:
X = processed_df.drop(['fully_funded','projectid','teacher_acctid','schoolid','school_ncesid'], axis=1)

X = X.select_dtypes(exclude=['object', 'bool'])
y = processed_df['fully_funded']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=100000, test_size=0.2,
    random_state=42, stratify=y
    )

# Feature selection
selector = SelectKBest(f_classif, k=20)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Get selected feature names
selected_features = X.columns[selector.get_support()]
print("Selected features:", selected_features)

Selected features: Index(['school_latitude', 'school_charter', 'school_magnet', 'school_nlns',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support',
       'eligible_double_your_impact_match', 'eligible_almost_home_match',
       'primary_focus_area_Literacy & Language',
       'primary_focus_area_Math & Science',
       'primary_focus_area_Music & The Arts', 'school_metro_suburban',
       'school_metro_urban', 'grade_level_Grades 9-12',
       'poverty_level_highest poverty', 'poverty_level_moderate poverty',
       'resource_type_Supplies', 'resource_type_Technology'],
      dtype='object')


In [6]:
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd

processed_df['date_posted'] = pd.to_datetime(processed_df['date_posted'])
processed_df = processed_df.sort_values('date_posted')

tscv = TimeSeriesSplit(n_splits=5)
temporal_results = {}

In [ ]:
def train_and_evaluate_with_time_cv(model, X, y, time_indices):
    roc_scores = []
    fold = 1
    
    X_indices = X.index.intersection(time_indices)
    X_time = X.loc[X_indices]
    y_time = y.loc[X_indices]
    
    for train_idx, test_idx in tscv.split(X_time):
        X_train, X_test = X_time.iloc[train_idx], X_time.iloc[test_idx]
        y_train, y_test = y_time.iloc[train_idx], y_time.iloc[test_idx]
        
        X_train_selected = selector.transform(X_train)
        X_test_selected = selector.transform(X_test)
        
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        y_pred_proba = model.predict_proba(X_test_selected)[:, 1]
        
        roc_score = roc_auc_score(y_test, y_pred_proba)
        roc_scores.append(roc_score)
        
        print(f"Fold {fold} - ROC AUC: {roc_score:.4f}")
        print(classification_report(y_test, y_pred))
        print("Confusion Matrix:")
        print(confusion_matrix(y_test, y_pred))
        print("-" * 50)
        
        fold += 1
    
    avg_roc = np.mean(roc_scores)
    print(f"Average ROC AUC Score across all folds: {avg_roc:.4f}")
    
    return roc_scores, avg_roc

#### Run

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=100000),
    'Naive Bayes': GaussianNB(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': xgb.XGBClassifier()
}

time_ordered_indices = processed_df.sort_values('date_posted').index

for name, model in models.items():
    print(f"\n{name} Results:")
    train_and_evaluate_with_time_cv(model, X, y, time_ordered_indices)

In [ ]:
time_ordered_indices = processed_df.sort_values('date_posted').index

for name, model in models.items():
    print(f"\nTemporal Cross-Validation Results for {name}:")
    scores, avg_score = train_and_evaluate_with_time_cv(model, X, y, time_ordered_indices)
    temporal_results[name] = {'scores': scores, 'avg_score': avg_score}

best_temporal_model = max(temporal_results.items(), key=lambda x: x[1]['avg_score'])[0]
print(f"\nBest model based on temporal cross-validation: {best_temporal_model}")

In [ ]:
plt.figure(figsize=(12, 6))

for name, result in temporal_results.items():
    if 'scores' in result and len(result['scores']) > 0:
        plt.plot(range(1, len(result['scores']) + 1), result['scores'], marker='o', label=name)

plt.title('ROC AUC Scores Across Time Series Folds')
plt.xlabel('Fold')
plt.ylabel('ROC AUC Score')
plt.xticks(range(1, 6))
plt.ylim(0.5, 1.0)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
temporal_stability = {}

for name, result in temporal_results.items():
    if 'scores' in result and len(result['scores']) > 0:
        scores = result['scores']
        std_dev = np.std(scores)
        stability_score = 1 / (1 + std_dev)  
        temporal_stability[name] = stability_score

if len(temporal_stability) > 0:
    plt.figure(figsize=(10, 5))
    plt.bar(list(temporal_stability.keys()), list(temporal_stability.values()))
    plt.title('Temporal Stability of Models')
    plt.xlabel('Model')
    plt.ylabel('Stability Score (higher is better)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    for name, stability in sorted(temporal_stability.items(), key=lambda x: x[1], reverse=True):
        print(f"{name}: {stability:.4f}")
else:
    print("No stability scores available. Run the temporal cross-validation first.")